In [13]:

import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Load the DenseNet121 model without the top (classification) layers
base_model = DenseNet121(weights=None, include_top=False, input_shape=(224, 224, 3))

# binary classification (hypertensive vs. normal)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Load the image filenames and labels from the Excel file
data = pd.read_excel('C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/2-Groundtruths/HRDC Hypertensive Retinopathy Classification Training Labels.xlsx', engine='openpyxl')
image_files = data['Image'].tolist()
labels = data['Hypertensive Retinopathy'].tolist()

data['Hypertensive Retinopathy'] = data['Hypertensive Retinopathy'].astype(str)

# Create data generator for training and validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory='C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/1-Images/1-Training Set',
    x_col='Image',
    y_col='Hypertensive Retinopathy',
    subset='training',
    batch_size=32,
    shuffle=True,
    class_mode='binary',
    target_size=(224, 224)
)

valid_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory='C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/1-Images/1-Training Set',
    x_col='Image',
    y_col='Hypertensive Retinopathy',
    subset='validation',
    batch_size=32,
    shuffle=True,
    class_mode='binary',
    target_size=(224, 224)
)

# Train the model
model.fit(train_generator, validation_data=valid_generator, epochs=100)

# Evaluate the model on the test set
test_generator = datagen.flow_from_dataframe(
    dataframe=data,
    directory='C:/Users/DELL/Documents/IAS internship docs/IAS project/Hypertensive retinopathy/Dataset/2-Hypertensive Retinopathy Classification/1-Images/1-Training Set',
    x_col='Image',
    y_col='Hypertensive Retinopathy',
    batch_size=32,
    shuffle=False,
    class_mode='binary',
    target_size=(224, 224)
)

loss, accuracy = model.evaluate(test_generator)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)


Found 570 validated image filenames belonging to 2 classes.
Found 142 validated image filenames belonging to 2 classes.
Epoch 1/20
18/18 [==============================] - 406s 19s/step - loss: 0.7790 - accuracy: 0.6175 - val_loss: 0.7366 - val_accuracy: 0.2817
Epoch 2/20
18/18 [==============================] - 304s 17s/step - loss: 0.6887 - accuracy: 0.6544 - val_loss: 0.7925 - val_accuracy: 0.2606
Epoch 3/20
18/18 [==============================] - 302s 17s/step - loss: 0.6558 - accuracy: 0.6544 - val_loss: 0.7034 - val_accuracy: 0.4930
Epoch 4/20
18/18 [==============================] - 293s 16s/step - loss: 0.6413 - accuracy: 0.6632 - val_loss: 0.7176 - val_accuracy: 0.3592
Epoch 5/20
18/18 [==============================] - 2085s 122s/step - loss: 0.6314 - accuracy: 0.6754 - val_loss: 0.7343 - val_accuracy: 0.2606
Epoch 6/20
18/18 [==============================] - 287s 16s/step - loss: 0.6251 - accuracy: 0.6667 - val_loss: 0.8206 - val_accuracy: 0.2606
Epoch 7/20
18/18 [========